In [1]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import time

from config import *

In [2]:
def search_song(artist, title):
    '''
    Query spotify API for a given song to obtain its URI.
    
    Args:
        artist: Artist of the song
        title: Song title
    Return:
        On success the URI is returned, on error "invalid"
    '''
    try:
        print(f'Query for "{title}" by "{artist}": ', end='')
        result = sp.search(q=f'track:{title} artist:{artist}', limit=1)
        uri = result['tracks']['items'][0]['uri']
        print(uri)
        return uri
    except:
        print('failed')
        return "invalid"

In [3]:
def get_audio_features(chunk):
    '''
    Get audiofeatures for given dataframe chunk and return them
    as a dictionary.
    
    Args:
        chunk: Pandas dataframe
    Return:
        Dictionary with audiofeatures
    '''

    d = {'danceability': [],
         'energy': [],
         'key': [],
         'loudness': [],
         'mode': [],
         'speechiness': [],
         'acousticness': [],
         'instrumentalness': [],
         'liveness': [],
         'valence': [],
         'tempo': [],
         'type': [],
         'id': [],
         'uri': [],
         'track_href': [],
         'analysis_url': [],
         'duration_ms': [],
         'time_signature': []
        }
    
    for index, row in chunk.iterrows():

        uri = search_song(row['artist'], row['title'])       
        audio_feat = sp.audio_features(uri)[0]        

        if audio_feat == None:
            # If we couldn't receive the audio features from spotify,
            # just store a 'x' for each feature and drop the song later...
            for key in d:
                d[key].append('x')
        else:        
            d['danceability'].append(audio_feat['danceability'])
            d['energy'].append(audio_feat['energy'])
            d['key'].append(audio_feat['key'])
            d['loudness'].append(audio_feat['loudness'])
            d['mode'].append(audio_feat['mode'])
            d['speechiness'].append(audio_feat['speechiness'])
            d['acousticness'].append(audio_feat['acousticness'])
            d['instrumentalness'].append(audio_feat['instrumentalness'])
            d['liveness'].append(audio_feat['liveness'])
            d['valence'].append(audio_feat['valence'])
            d['tempo'].append(audio_feat['tempo'])
            d['type'].append(audio_feat['type'])
            d['id'].append(audio_feat['id'])
            d['uri'].append(audio_feat['uri'])
            d['track_href'].append(audio_feat['track_href'])
            d['analysis_url'].append(audio_feat['analysis_url'])
            d['duration_ms'].append(audio_feat['duration_ms'])
            d['time_signature'].append(audio_feat['time_signature'])

        time.sleep(0.5)
    return d

In [4]:
def chunk_dataframe(df, chunk_size=50):
    '''
    Slices a pandas dataframe into chunks of given size
    and return a list containing all these chunks.
    
    Args:
        df: Pandas dataframe to slice
        chunk_size: Size of single chunk (Default=50)
    Return:
        List with dataframe chunks
    '''
    start  = 0
    end    = chunk_size
    chunks = []

    while start < len(df):
        chunks.append(df.iloc[start:end, :])
        start = end
        end   = end+chunk_size
        
    return chunks

In [5]:
def get_dataframe_audio_features(df):
    '''
    Download audio features for all songs within the given pandas dataframe.
    Note: Given df MUST have a title and artist column!!!
    
    Args:
        df: Pandas dataframe with songs (artist and title)
    Return:
        Pandas dataframe with songs and its related audio features
    '''

    df_total = pd.DataFrame()
    chunks   = chunk_dataframe(df)
    n_chunk  = 1
    
    for chunk in chunks:

        chunk = chunk.reset_index(drop=True)
        
        # 
        print(f'-> Downloading audio-features for chunk {n_chunk}')
        
        # Get audio features for current chunk
        audio_feat = get_audio_features(chunk)
        
        # Create df = chunk + audio_features
        chunk_with_feat = pd.concat([chunk, pd.DataFrame(audio_feat)], axis=1)

        # Add to total df
        if len(df_total) == 0:
            df_total = chunk_with_feat
        else:
            df_total = pd.concat([df_total, chunk_with_feat], axis=0)

        # Wait for 20 secs, for don't get banned
        time.sleep(20)
        n_chunk += 1

    return df_total

In [6]:
# Establish connection to spotify API.
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))
# Get audio features for songs provided in 'hot.csv'
hot_df = pd.read_csv('hot.csv')
hot_with_features_df = get_dataframe_audio_features(hot_df)
hot_with_features_df = hot_with_features_df[hot_with_features_df['key'] != 'x']
hot_with_features_df.to_csv('hot_with_features.csv', index=False)

# Get audio features for songs provided in 'nothot.csv'
nothot_df = pd.read_csv('nothot.csv')
nothot_with_features_df = get_dataframe_audio_features(nothot_df)
nothot_with_features_df = nothot_with_features_df[nothot_with_features_df['key'] != 'x']
nothot_with_features_df.to_csv('nothot_with_features.csv', index=False)

-> Downloading audio-features for chunk 1
Query for "Anti-Hero" by "Taylor Swift": spotify:track:0V3wPSX9ygBnCm8psDIegu
Query for "Lavender Haze" by "Taylor Swift": spotify:track:5jQI2r1RdgtuT8S3iG8zFC
Query for "Maroon" by "Taylor Swift": spotify:track:3eX0NZfLtGzoLUxPNvRfqm
Query for "Snow On The Beach" by "Taylor Swift Featuring Lana Del Rey": failed
Query for "Midnight Rain" by "Taylor Swift": spotify:track:3rWDp9tBPQR9z6U5YyRSK4
Query for "Bejeweled" by "Taylor Swift": spotify:track:0jvo9CjnbR0lYUDTSNTMiu
Query for "Question...?" by "Taylor Swift": spotify:track:6C7OF1Jal7YkyjzyUG8vkK
Query for "You're On Your Own, Kid" by "Taylor Swift": failed
Query for "Karma" by "Taylor Swift": spotify:track:7KokYm8cMIXCsGVmUvKtqf
Query for "Vigilante Shit" by "Taylor Swift": spotify:track:1xwAWUI6Dj0WGC3KiUPN0O
Query for "Unholy" by "Sam Smith & Kim Petras": failed
Query for "Bad Habit" by "Steve Lacy": spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK
Query for "Mastermind" by "Taylor Swift": spotify:tra

Query for "(Everything I Do) I Do it For You" by "Bryan Adams": spotify:track:6eBK3edMW7bEzecF1eCezc
Query for "I Will Always Love You" by "Whitney Houston": spotify:track:4eHbdreAnSOrDDsFfc4Fpm
Query for "Another Brick in the Wall (part 2)" by "Pink Floyd": spotify:track:1xUTI8Ce8IQnyYCGPmrbzS
Query for "Flashdance... What a Feeling" by "Irene Cara": spotify:track:5CGziaEatzKez145qf7Xj2
Query for "Candle in the Wind '97" by "Elton John": failed
Query for "(I Can't Get No) Satisfaction" by "The Rolling Stones": spotify:track:2PzU4IB8Dr6mxV3lHuaG34
Query for "Gangsta's Paradise" by "Coolio": spotify:track:1DIXPcTDzTj8ZMHt3PDt8p
Query for "Lose Yourself" by "Eminem": spotify:track:5Z01UMMf7V1o0MzF86s6WJ
Query for "Believe" by "Cher": spotify:track:2goLsvvODILDzeeiT4dAoR
Query for "Yeah!" by "Usher": spotify:track:5rb9QrpfcKFHM1EUbSIurX
Query for "Let it Be" by "The Beatles": spotify:track:7iN1s7xHE4ifF5povM6A48
Query for "A Whiter Shade of Pale" by "Procol Harum": spotify:track:78ZqE2tjA

Query for "Upside Down" by "Diana Ross": spotify:track:3pbtBomO4Zt5gGiqsYeiBH
Query for "Music" by "Madonna": spotify:track:2BcOZ4vOeu2qZowIHrvTYy
Query for "How You Remind Me" by "Nickelback": spotify:track:0gmbgwZ8iqyMPmXefof8Yf
Query for "Sugar Sugar" by "Archies": spotify:track:6MTd61g9zq6CB1FnJydjEb
Query for "Lady Marmalade (Voulez-Vous Coucher Aver Moi Ce Soir?)" by "Christina Aguilera, Lil' Kim, Mya & Pink": failed
Query for "Take On Me" by "A-Ha": spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
Query for "The Ketchup Song (Asereje)" by "Las Ketchup": spotify:track:6NG2IIXZfC5Notpz0GIODU
Query for "Get Back" by "The Beatles": spotify:track:4MLBqAEzNN89o2M9h92Z26
Query for "Smooth" by "Rob Thomas & Santana": spotify:track:5IALWUYK0zDSEmZgb4ICvc
Query for "Informer" by "Snow": spotify:track:2LjiPAQOVazT8sRyXL3XRs
Query for "Apologize" by "Timbaland & OneRepublic": spotify:track:6EibZ5dWHFYzXgxbEjNvat
Query for "All the Things She Said" by "t.A.T.u.": spotify:track:4bJygwUKrRgq1stlNXcgMg
Que

Query for "Hot 'n' cold" by "Katy Perry": spotify:track:1y4eb6hmAvsqlDOl3fx9kk
Query for "Waterfalls" by "TLC": spotify:track:6qspW4YKycviDFjHBOaqUY
Query for "It Must Have Been Love" by "Roxette": spotify:track:6kvoHl80mfCVTv7XnZkjQn
Query for "Tragedy" by "Bee Gees": spotify:track:6UXXeFqMBGiqjkzQzkMT3E
Query for "A Hard Day's Night" by "The Beatles": spotify:track:5J2CHimS7dWYMImCHkEFaJ
Query for "La Bamba" by "Los Lobos": spotify:track:0uMMLry3hzWGn3q3loqMkm
Query for "Mister Sandman" by "The Chordettes": spotify:track:17CKZQwWeBWiFP9nKjWmbJ
Query for "Maggie May" by "Rod Stewart": spotify:track:6rovOdp3HgK1DeAMYDzoA7
Query for "A Groovy Kind of Love" by "Phil Collins": spotify:track:5OHUTC4EvTbL0DuZQRIGGX
Query for "In the Year 2525 (Exordium & Terminus)" by "Zager & Evans": spotify:track:5UVsbUV0Kh033cqsZ5sLQi
Query for "Fantasy" by "Mariah Carey": spotify:track:6xkryXuiZU360Lngd4sx13
Query for "Faith" by "George Michael": spotify:track:0HEmnAUT8PHznIAAmVXqFJ
Query for "I Want to

Query for "Bye, Bye, Bye" by "N Sync": spotify:track:62bOmKYxYg7dhrC6gH9vFn
Query for "I Feel Love" by "Donna Summer": spotify:track:7B7lf3sIze5VR2WuYttn18
Query for "All Night Long (All Night)" by "Lionel Richie": spotify:track:4giIdAr7Plyu5LQzBOR4FP
Query for "White Flag" by "Dido": spotify:track:3adnLFXKO5rC1lhUNSeg3N
Query for "Sentimental Journey" by "Les Brown & Doris Day": spotify:track:3RUsMlbbqWzCugah3BtWZT
Query for "Mr Tambourine Man" by "The Byrds": spotify:track:2HCaIYjkvWSZzaSKUoOh3d
Query for "Baker Street" by "Gerry Rafferty": spotify:track:5gOd6zDC8vhlYjqbQdJVWP
Query for "On The Floor" by "Jennifer Lopez & Pitbull": spotify:track:48YAqrcjQqPZSF7k4yOss4
Query for "All Shook Up" by "Elvis Presley": spotify:track:5ueyLj6e6oVaTY0KQ6yLaA
Query for "Ticket to Ride" by "The Beatles": spotify:track:7CZiDzGVjUssMSOXrDNYHL
Query for "Torn" by "Natalie Imbruglia": spotify:track:1Jaah2tmN9Hv81A87KZ1MU
Query for "Losing My Religion" by "REM": spotify:track:45n3sfgLdzVAuiCVS47a7h
Q

...

Query for "Get Down On It" by "Kool & The Gang": spotify:track:2je56j0xuoTi1gYkLLMlJU
Query for "Someday" by "Nickelback": spotify:track:4PkJ7c9y1CwpuVOiJODnCZ
Query for "I'm Leaving it (All) Up to You" by "Donny Osmond & Marie Osmond": failed
Query for "Hazard" by "Richard Marx": spotify:track:1ntKYuqLlmV3sTZoNyeHhM
Query for "There You'll Be" by "Faith Hill": spotify:track:1s2khOWzC99udpUaPICLJI
Query for "On Our Own (Theme from 'Ghostbusters II')" by "Bobby Brown": failed
Query for "Blinded By the Light" by "Manfred Mann's Earth Band": spotify:track:7kNNylJ1kswWbHeRM6UDuE
Query for "Shannon" by "Henry Gross": spotify:track:4IYoyOVK9oQyJS1QZt5Vyj
Query for "Mam'selle" by "Frank Sinatra": failed
Query for "Ashes to Ashes" by "David Bowie": spotify:track:37CRPk0L5ZpfPeePEPwE0t
Query for "China Girl" by "David Bowie": spotify:track:5Ei9o2fHEnfHb84YrPuz4v
Query for "Stop Stop Stop" by "The Hollies": spotify:track:3Jw2A9SC8zhntx4ON9VabX
Query for "Limbo Rock" by "Chubby Checker": spotify:

Query for "The Way You Move" by "OutKast": spotify:track:5z7mYFfhw6N6f23VwrokJD
Query for "El Paso" by "Marty Robbins": spotify:track:4f8hBeMXMvssn6HtFAtblo
Query for "Magic" by "Pilot": spotify:track:500h8jAdr7LvzzXlm1qxtK
Query for "Hate That I Love You" by "Rihanna": spotify:track:7iu0WYLdo4yksKf3seaxzI
Query for "The Hindu Times" by "Oasis": spotify:track:7IuCCezGsyTH9LU4IVL57s
Query for "I Wanna Be Loved By You" by "Helen Kane": spotify:track:0birwxao4QdkCd8IUPloCR
Query for "Fergalicious" by "Fergie & will.i.am": spotify:track:3QmlTX1DiAxgQpSIobzce6
Query for "Roses" by "OutKast": spotify:track:6bUNEbXT7HovLW6BgPCBsb
Query for "All Along the Watchtower" by "Jimi Hendrix": spotify:track:2aoo2jlRnM3A0NyLQqMN2f
Query for "Just Like a Pill" by "Pink": failed
Query for "Hey Baby" by "Bruce Channel": spotify:track:0obIo2GEab6qqms6t3JaMY
Query for "Good Time" by "Owl City & Carly Rae Jepsen": spotify:track:0V1MeRzoASQXtFZU3qt71e
Query for "Shattered Dreams" by "Johnny Hates Jazz": spoti

Query for "Don't it Make My Brown Eyes Blue" by "Crystal Gayle": failed
Query for "Valleri" by "The Monkees": spotify:track:2U1nVCTFKFdieTPTg3VrvY
Query for "Dance to the Music" by "Sly & The Family Stone": spotify:track:1MQWtVcs0PKsY4PA6ZvLiy
Query for "Mr Saxobeat" by "Alexandra Stan": spotify:track:2ZAx6NDhX2yE5aL5ueEQgY
Query for "Wedding Bell Blues" by "Fifth Dimension": failed
Query for "The Streak" by "Ray Stevens": spotify:track:3HiIBzaoBkOTLmRYkdsusv
Query for "Free As a Bird" by "The Beatles": spotify:track:6hLY3Tz1Xt5kBuKNDTs4ib
Query for "Vienna" by "Ultravox": spotify:track:2LXjhNee4iPRVemx9NSZQ3
Query for "The Heat is On" by "Glenn Frey": spotify:track:0PXw9NKvolWTo7U9JkNzmc
Query for "Rock the Casbah" by "The Clash": spotify:track:56KqaFSGTb7ifpt16t5Y1N
Query for "Smooth Criminal" by "Alien Ant Farm": spotify:track:5z6xHjCZr7a7AIcy8sPBKy
Query for "Ride On Time" by "Black Box": spotify:track:3pgainA2dH9c7e8JVYGRCN
Query for "It's Love-Love-Love" by "Guy Lombardo": spotif

Query for "Little Arrows" by "Leapy Lee": spotify:track:6thVOj1grfYYEvru4PSP2J
Query for "The Message" by "Grandmaster Flash & The Furious Five": spotify:track:5DuTNKFEjJIySAyJH1yNDU
Query for "In Dreams" by "Roy Orbison": spotify:track:44pN9jfSqY3d3lrKuv3QeN
Query for "Cruising Down the River" by "Russ Morgan": spotify:track:7bVD6VV422a8Y9PqfCksWG
Query for "Son of a Preacher Man" by "Dusty Springfield": spotify:track:7odHgoLFi3GQ90E9PeraI3
Query for "I'm Gonna Getcha Good!" by "Shania Twain": failed
Query for "Too Fat Polka (I Don't Want Her-You Can Have Her-She's Too Fat For Me)" by "Arthur Godfrey": failed
Query for "Stay (Faraway, So Close!)" by "U2": spotify:track:1E1ylgIWTW2yE0glywYLb6
Query for "Tumbling Dice" by "The Rolling Stones": spotify:track:4hq0S6wznq7SHDyMOFXL9i
Query for "Bad Medicine" by "Bon Jovi": spotify:track:72hcFp4tYkd3dbNA9dZ3Pv
Query for "Let Her Go" by "Passenger": spotify:track:1KxwZYyzWNyZSRyErj2ojT
Query for "Doctor, Lawyer, Indian Chief" by "Betty Hutton

Query for "How Do You Do!" by "Roxette": spotify:track:3BK8mbUFbBZ3mI5HE4eqCR
Query for "China in Your Hand" by "T'Pau": failed
Query for "Needles & Pins" by "The Searchers": spotify:track:0zABw1PtGYHsU4VDpDZsra
Query for "Have a Nice Day" by "Bon Jovi": spotify:track:06Zz2l7cVTCTgCNDe1bnEF
Query for "Morning Has Broken" by "Cat Stevens": spotify:track:5y8d8RHvxb0KiTa9Nq0xWp
Query for "The Love You Save" by "The Jackson 5": spotify:track:2dYdai0GywuwYq6blSwDBV
Query for "You've Made Me So Very Happy" by "Blood Sweat & Tears": spotify:track:3fYFfrObiL72Ud3Py5wshL
Query for "It's Not Unusual" by "Tom Jones": spotify:track:3TlIt0ReIxPsVZcOEivT5U
Query for "From Me to You" by "The Beatles": spotify:track:1euKWJfJSfPhgSendPrcXb
Query for "Rudolph, the Red-Nosed Reindeer" by "Bing Crosby": spotify:track:0XUARKkBvikXDEjSzdkWFP
Query for "Aline" by "Christophe": spotify:track:1N4ixxhbBH1ClnPdTTsRzz
Query for "I'm Still Standing" by "Elton John": failed
Query for "The Closer I Get To You" by "R